In [1]:
import os
from time import time, localtime
import logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import numpy as np
# from utils import plot_graph
# import tensorflow_federated as tff
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras import losses, metrics, optimizers
import random
import pandas as pd
import sys
import matplotlib.pyplot as plt
import nest_asyncio
from pathlib import Path
# from tensorflow_federated.python.research.utils import checkpoint_manager
# import tensorflow_addons as tfa
nest_asyncio.apply()
SEED = 1337
tf.random.set_seed(SEED)

In [2]:
def most_frequent(List):
    return max(set(List), key=List.count)

In [3]:
test_dir = "/home/jovyan/dataset/GQUIC_small/Test/GQUIC_test_32.feather"
test = pd.read_feather(test_dir)
# print(test)
test.to_csv("/home/jovyan/dataset/GQUIC_small/Test/GQUIC_test_32.csv")
print(test)


       flow_id    0    1    2    3    4    5    6    7    8  ...   23   24  \
0           29   16   10  149  182  192  172  158  109   13  ...  231  253   
1           29   16   10  168  155  186  253  154  217  174  ...  154  238   
2           29   16   10  167  101   83  237  126   84  226  ...  119  108   
3           29   16   10  166  230  173   64    4  169   36  ...  248   79   
4           29   16   10  165   59   89  192  140  124  161  ...  235   26   
...        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
55595   138365  144  111   86  255  137  119   88  101  151  ...   27    2   
55596   138365  144  239   86  254  137  119   84  165  151  ...   88  189   
55597   138365  144  111   87   16  137  119  152   37  151  ...   46  104   
55598   138365  144  111   87    6  137  119  114  165  151  ...  170   76   
55599   138365  144  111   87   17  137  119  155  229  151  ...  124  166   

        25   26   27   28   29   30   31  Label  
0       40   

In [4]:
result = test.groupby('flow_id')['Label'].apply(list).to_dict()
# print(result, type(result))
flow_label = []
for flow in result:
    flow_label.append(most_frequent(result[flow]))

flow_label = np.array(flow_label)

In [5]:
true_test = test.drop('flow_id', axis=1)
# print(true_test)

In [6]:
x_test = true_test.drop('Label', axis=1).to_numpy()/255
print(x_test)

[[0.0627451  0.03921569 0.58431373 ... 0.20392157 0.04705882 0.6       ]
 [0.0627451  0.03921569 0.65882353 ... 0.70196078 0.44313725 0.43137255]
 [0.0627451  0.03921569 0.65490196 ... 0.89411765 0.16470588 0.74117647]
 ...
 [0.56470588 0.43529412 0.34117647 ... 0.49411765 0.69019608 0.23137255]
 [0.56470588 0.43529412 0.34117647 ... 0.74901961 0.04313725 0.74509804]
 [0.56470588 0.43529412 0.34117647 ... 0.68627451 0.67058824 0.83137255]]


In [7]:
x_test[20]

array([0.0627451 , 0.04705882, 0.49411765, 0.31372549, 0.51372549,
       0.43137255, 0.05098039, 0.80392157, 0.57647059, 0.69411765,
       0.3372549 , 0.96470588, 0.08627451, 0.79215686, 0.44705882,
       0.7254902 , 0.78823529, 0.97254902, 0.00784314, 0.09803922,
       0.6       , 0.64313725, 0.11372549, 0.18039216, 0.63137255,
       0.22352941, 0.70980392, 0.11764706, 0.75686275, 0.16470588,
       0.79607843, 0.1372549 ])

In [8]:
print(x_test.shape)
x_test = x_test.reshape(-1,20,128,1)
print(x_test.shape)

(55600, 32)
(695, 20, 128, 1)


In [9]:
x_test[1]

array([[[0.0627451 ],
        [0.09411765],
        [0.1372549 ],
        ...,
        [0.14509804],
        [0.88627451],
        [0.94117647]],

       [[0.0627451 ],
        [0.09411765],
        [0.15686275],
        ...,
        [0.05490196],
        [0.66666667],
        [0.39607843]],

       [[0.0627451 ],
        [0.09411765],
        [0.14117647],
        ...,
        [0.96470588],
        [0.55294118],
        [0.71372549]],

       ...,

       [[0.04705882],
        [0.45490196],
        [0.98823529],
        ...,
        [0.54117647],
        [0.60392157],
        [0.1254902 ]],

       [[0.04705882],
        [0.45490196],
        [0.98823529],
        ...,
        [0.80784314],
        [0.77254902],
        [0.74509804]],

       [[0.        ],
        [0.76078431],
        [0.9372549 ],
        ...,
        [0.51764706],
        [0.43137255],
        [0.48235294]]])

### Model Definition

In [10]:
model = tf.keras.models.load_model("/home/jovyan/dataset/model_2")

In [11]:
model

In [12]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 20, 128, 32)       832       
                                                                 
 conv2d_13 (Conv2D)          (None, 20, 128, 32)       25632     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 10, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 10, 64, 64)        18496     
                                                                 
 conv2d_15 (Conv2D)          (None, 10, 64, 64)        36928     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 5, 32, 64)        0         
 2D)                                                  

### Predict

#### Predict all

In [13]:
predictions = model.predict(x_test,verbose=2,use_multiprocessing=True,batch_size=64)
print(predictions)

11/11 - 1s - 706ms/epoch - 64ms/step
[[1.0626294e-08 9.9896431e-01 1.4558329e-06 1.0342860e-03]
 [7.0241816e-07 9.5604706e-01 1.2743822e-04 4.3824881e-02]
 [2.0174480e-06 6.6617328e-01 8.0142915e-04 3.3302328e-01]
 ...
 [1.7344541e-04 9.9575979e-01 3.1113498e-06 4.0637380e-03]
 [2.3151218e-04 9.6708393e-01 2.3404445e-04 3.2450531e-02]
 [9.7959220e-02 6.6721952e-01 7.1256296e-03 2.2769560e-01]]


In [14]:
flow_pred = np.argmax(predictions, axis=-1)

In [15]:
from notebook.services.config import ConfigManager
cm = ConfigManager().update('notebook', {'limit_output': 1000})

In [16]:
one_flow = x_test[0].reshape(-1,20,128,1)
print(one_flow)

predictions_1flow = model.predict(one_flow)
one_flow_pred = int(np.argmax(predictions_1flow, axis=-1))
print("Label", one_flow_pred)

[[[[0.0627451 ]
   [0.03921569]
   [0.58431373]
   ...
   [0.07843137]
   [0.49803922]
   [0.23921569]]

  [[0.0627451 ]
   [0.03921569]
   [0.64705882]
   ...
   [0.43529412]
   [0.48235294]
   [0.1372549 ]]

  [[0.0627451 ]
   [0.03921569]
   [0.62745098]
   ...
   [0.18431373]
   [0.83137255]
   [0.31764706]]

  ...

  [[0.0627451 ]
   [0.00392157]
   [0.02352941]
   ...
   [0.56078431]
   [0.41568627]
   [1.        ]]

  [[0.0627451 ]
   [0.        ]
   [0.99215686]
   ...
   [0.40392157]
   [0.56078431]
   [0.43137255]]

  [[0.0627451 ]
   [0.00392157]
   [0.00392157]
   ...
   [0.98039216]
   [0.30980392]
   [0.98823529]]]]
1/1 [==============================] - 0s 18ms/step
Label 1


In [ ]:
label_dict ={'FileTransfer': 0, 'Music': 1, 'VoIP': 2, 'Youtube': 3}

In [19]:
for key, value in label_dict.items():
    if one_flow_pred == value:
        print(key)


Music


In [20]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report


print('\nAccuracy: {:.2f}\n'.format(accuracy_score(flow_label, flow_pred)))

ValueError: Found input variables with inconsistent numbers of samples: [2780, 695]